<a href="https://colab.research.google.com/github/zhangxs131/Spider/blob/main/heritage_spider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#requests+lxml 爬取外交关系委员会 https://www.heritage.org/

该网站为静态网站，直接requests+lxml 进行爬取

*       搜索词为querys数组
*       start_time 为具体新闻开始时间设置
*       end_time  新闻结束时间设置

这里使用线程池加快爬取，我这里写的多线程有些问题，待修改


In [18]:
import requests
import lxml.etree as etree
import datetime
import json
import os
from concurrent.futures import ThreadPoolExecutor
import threading

pool = ThreadPoolExecutor(max_workers=10)



querys=['Ukraine','Russia','punish','politics','military','sports','world']
pp='10'
pages=20
start_time='2022-03-13'
end_time='2022-03-17'


ori_url='https://www.heritage.org/search?contains={}&type%5B%5D=commentary&type%5B%5D=report&range_start={}&range_end={}&date_offset=&page={}'

result={}
sum=0

def thread_action(li):
    sample={}
  # try:
    sample['title']=li.xpath('.//a[@class="result-card__title js-hover-target"]/text()')[0].strip('\n')
    link=li.xpath('.//a[@class="result-card__title js-hover-target"]/@href')
    #获取文章页面
    page_url='https://www.heritage.org'+link[0]
    print('URL',page_url)
    page_res=requests.get(page_url)
    t=etree.HTML(page_res.text) 
    sample['text']=t.xpath('//div[@class="article__body-copy"]//p//text()')
    sample['url']=page_url
    if len(sample['text'])==0:
      sample['text']=t.xpath('//section[@class="article__body-copy"]//p//text()')
      if len(sample['text'])==0:
        return {}
      
    print('title: ',sample['title'])
    sample['text']=('').join(sample['text'])
    print('text',sample['text'])
    this_time=li.xpath('.//p[@class="result-card__date"]/span/text()')[0]

    sample['time']=str(datetime.datetime.strptime(this_time,'%b %d, %Y'))
    print(sample['time'])
    # sample['time']= datetime.datetime.strptime(('-').join(this_time[0].split('-')[:-1]),'%Y-%m-%dT%H:%M:%S')

    return sample
  # except:
  #   print('error in pages')
  #   return {}

for query in querys:
  print('\n\n____________________________under searching keywork {}_________________________________'.format(query))

  data=[]
  for i in range(pages):
    url=ori_url.format(query,start_time,end_time,i)
    res=requests.get(url)
    print('Page {} ______________________________url :{}'.format(i+1,url))

    #防止超出序列
    lis = etree.HTML(res.text).xpath('//div[@class="views-row"]')
    if len(lis)==0:
      break

    for j in lis:
      #多线程
      future= pool.submit(thread_action, j)

      if future.result()=={}:
        continue
      sum+=1
      if sum%20==0:
        print('___________________________Sum: {} News Time: {} ______________________'.format(sum,future.result()['time']))

      data.append(future.result())

    if future.result()!={} and future.result()['time']<start_time:
      break


  result[query]=data
  data=[]

pool.shutdown()

#save data
with open('heritage.json'.format(query),'w') as f:
  f.write(json.dumps(result))




____________________________under searching keywork Ukraine_________________________________
Page 1 ______________________________url :https://www.heritage.org/search?contains=Ukraine&type%5B%5D=commentary&type%5B%5D=report&range_start=2022-03-13&range_end=2022-03-17&date_offset=&page=0
URL https://www.heritage.org/defense/commentary/putins-dangerous-nuclear-brinkmanship-ukraine
title:  Putin’s Dangerous Nuclear Brinkmanship in Ukraine
text Most Americans view nuclear weapons as a relic of the Cold War. President Ronald Reagan and Soviet leader Mikhail Gorbachev agreed in 1985 “that a nuclear war can never be won and must never be fought.” That thinking has remained a “given” for the last 37 years. Unfortunately, Russian President Vladimir Putin may have other ideas.Just five days before Russia invaded Ukraine, Putin personally oversaw nuclear exercises and issued subtle nuclear threats against the West should Ukraine become a part of NATO. Once his war begun, the threats became less